In [94]:
import numpy as np
import pandas as pd
import random
import time

In [95]:
n=100
#Create empty matrix A
A = np.zeros((n,n))

In [96]:
#Create y
np.random.seed(1)
y = np.random.choice([0,1,2], size=(n,1), p=[0.4,0.3,0.3])

In [97]:
#Create X
np.random.seed(2)
X = np.random.uniform(0.0, 1.0, (n,1))

In [98]:
#Change value of X
np.random.seed(3)
for i in range(0,n):
    if y[i] == 0:
        X[i,:] = np.random.beta(1,3)
    elif y[i] == 1:
        X[i,:] = np.random.beta(1,5)
    else:
        X[i,:] = np.random.beta(1,7)

In [99]:
#Create P
P=np.matmul(X,X.transpose())

In [100]:
P = P/np.amax(np.amax(P))

In [101]:
np.random.seed(5)
for i in range(0,n):
    for j in range(1,n):
        A[i,j]=np.random.binomial(2,P[i,j])

In [102]:
for i in range(0,n):
    for j in range(0,i):
        A[i,j] = A[j,i]
        A[i,i]=0
A

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Split Data

In [103]:
from sklearn.model_selection import RepeatedKFold
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
for train_index, test_index in rkf.split(A):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = A[train_index], A[test_index]
     y_train, y_test = y[train_index], y[test_index]
     X1 = X_train.transpose()
     X_train, X_test = X1[train_index], X1[test_index]

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 11 13 14 15 16 18 19 21 22 24 25 26 27 29
 30 31 33 36 38 39 41 43 44 45 47 48 49 50 51 52 53 54 55 56 58 59 61 62
 63 64 65 67 68 71 72 73 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90
 91 92 93 94 95 96 98 99] TEST: [10 12 17 20 23 28 32 34 35 37 40 42 46 57 60 66 69 70 74 97]
TRAIN: [ 0  1  2  3  4  5  7  8  9 10 11 12 14 15 16 17 18 19 20 23 27 28 29 30
 31 32 33 34 35 36 37 38 40 41 42 43 44 45 46 47 49 51 52 53 57 58 59 60
 61 62 65 66 67 68 69 70 73 74 75 76 77 78 79 80 81 82 83 84 85 86 89 90
 91 92 93 95 96 97 98 99] TEST: [ 6 13 21 22 24 25 26 39 48 50 54 55 56 63 64 71 72 87 88 94]
TRAIN: [ 0  1  2  4  5  6  7  8  9 10 12 13 15 16 17 18 20 21 22 23 24 25 26 27
 28 30 31 32 34 35 36 37 38 39 40 41 42 44 46 47 48 50 51 53 54 55 56 57
 58 60 61 62 63 64 65 66 68 69 70 71 72 73 74 75 76 77 78 80 81 84 87 88
 90 91 92 93 94 96 97 99] TEST: [ 3 11 14 19 29 33 43 45 49 52 59 67 79 82 83 85 86 89 95 98]
TRAIN: [ 3  6  7  8 10 11 12 13 14 15 16

In [104]:
X_train.shape,X_test.shape

((80, 80), (20, 80))

# 1

In [105]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
begin = time.time() 
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train.ravel())
LDAacc = clf.score(X_test, y_test)
print(LDAacc)
time.sleep(1) 
end = time.time()
print(f"Total runtime of the program is {end - begin}") 

0.35
Total runtime of the program is 1.01432204246521


# 2

In [106]:
from scipy import linalg
U, s, VT = linalg.svd(A)
U.shape,  s.shape, VT.shape

((100, 100), (100,), (100, 100))

In [107]:
newx=[]
for c in range(1,10) :
   newx.append(U[:,0:c]*s[0:c]**0.5)

In [108]:
begin1 = time.time()
clf2 = LinearDiscriminantAnalysis()
SVDaccuracy=[]
for i in range(0,8):
    clf.fit(newx[i], y.ravel())
    SVDaccuracy.append(clf.score(newx[i], y.ravel()))
print(SVDaccuracy)
time.sleep(2) 
end1 = time.time()
print(f"Total runtime of the program is {end1 - begin1}")

[0.49, 0.53, 0.51, 0.5, 0.54, 0.59, 0.57, 0.6]
Total runtime of the program is 2.0381970405578613


# 3

In [109]:
#Create w
w = np.zeros((int(n/5*4),3))
for x in range(0,int(n/5*4)):
    i=int(y_train[x])
    w[x][i]=1

In [110]:
z_train=np.matmul(X_train,w)

In [111]:
z_test = np.matmul(X_test,w)

In [112]:
begin2 = time.time()
from sklearn.neural_network import MLPClassifier
clf3 = MLPClassifier(random_state=2, max_iter=5000)
clf3.fit(z_train, y_train.ravel())
ZYAcc = clf3.score(z_test, y_test)
print(ZYAcc)
time.sleep(3) 
end2 = time.time()
print(f"Total runtime of the program is {end2 - begin2}")

0.5
Total runtime of the program is 4.689073085784912


# 4

In [113]:
begin3 = time.time()
clf4 = LinearDiscriminantAnalysis()
clf4.fit(z_train,y_train.ravel())
ZYLDA = clf4.score(z_test, y_test.ravel())
print(ZYLDA)
time.sleep(4) 
end3 = time.time()
print(f"Total runtime of the program is {end3 - begin3}")

0.35
Total runtime of the program is 4.006830215454102
